In [25]:
from Designer import SST, show, sfround

![Beam](2PL-beam.svg)

In [2]:
L1 = 3000.
L2 = 5500.
Pd = 30.
Pl = 60.

In [3]:
Fy = 345.
E = 200000.
G = 77000.
phi = 0.9

In [4]:
DeltaR = 360.

In [5]:
# Case 8
P = Pl*1000.
l = 2*L2 + L1
a = L2
DeltaI = P*a*(3*l*l-4*a*a)/(24.*E)
Ireq = DeltaI/(l/DeltaR)
show('P,l,a,*1E6,Ireq')

P    = 60000       
l    = 14000       
a    = 5500        
Ireq = 825.6 * 1E6 


In [6]:
Mf = (1.25*Pd + 1.5*Pl)*L2/1000.
Mf

701.25

In [7]:
# choose W610x113

In [35]:
from math import pi, sqrt

def check_Mr(Ix,Sx,Zx,Iy,J,Cw,D,B,T,W,Dsg,
             E=200000.,G=77000.,Fy=345.,L=0.,omega2=1.0,Ireq=0.,Mf=0.,Vf=0.,display=False):
    """This only works for W Shapes with unstiffened webs."""
    phi = 0.9
    if display:
        print('Section Properties and Dimensions:')
        show('Dsg,D,B,T,W,*1E6,Ix,*1E3,Sx,Zx,*1E6,Iy,*1E3,J,*1E9,Cw,*,E,G,Fy',data=locals())
        print('\nBeam Data:')
        show('*1E6,Ireq,*,Mf,L,omega2',data=locals())
    if Ix < Ireq:
        return False
    
    btlim = [x/sqrt(Fy) for x in [145,170,200]]
    hwlim = [x/sqrt(Fy) for x in [1100,1700,1900]]
    fclass = wclass = 4
    bt = B/(2*T)
    for i in range(1,4):
        if bt <= btlim[i-1]:
            fclass = i
            break
    hw = (D-2*T)/W
    for i in range(1,4):
        if hw <= hwlim[i-1]:
            wclass = i
            break
    sclass = max(fclass,wclass)
    
    if display:
        print('\nSection Class:')
        show('b/2t=bt,class 1=btlim[0],class 2=btlim[1],class 3=btlim[2],flange class=fclass',data=locals())
        show('h/w=hw,class 1=hwlim[0],class 2=hwlim[1],class 3=hwlim[2],web class=wclass,sect. class=sclass',data=locals())

    if sclass > 3:
        return False

    if sclass <= 2:
        Mpy = Mp = Fy*Zx * 1E-6
    else:
        Mpy = My = Fy*Sx * 1E-6
        
    if L > 0.:
        P = E*Iy*G*J
        Q = Iy*Cw*((pi*E)**2)
        Lu = 0.06742*(omega2**0.5)*((103684.0*((Mpy*1E6)**2)*Q + 55516.*(omega2**2)*(P**2))**0.5 + 235.62*omega2*P)**0.5/(Mpy*1E6)
        Mu = (omega2*pi/L)*sqrt(P+Q/(L**2)) * 1E-6
        if Mu > 0.67*Mpy:
            Mr = min(1.15*phi*Mpy*(1-0.28*Mpy/Mu),phi*Mpy)
        else:
            Mr = phi*Mu
    else:
        Mr = phi*Mpy
        
    if display:
        if sclass < 3:
            s = 'Mp=Mpy,phi*Mp,L,Lu,Mu,0.67*Mp=0.67*Mpy,Mr'
        else:
            s = 'My=Mpy,phi*My,L,Lu,Mu,0.67*My=0.67*Mpy,Mr'
        print('\nMoment Strength:')
        show(s,data=locals())
        
    if Mr < Mf:
        return False
    
    hw = (D-2*T)/W
    Aw = D*W
    if hw <= 1014./sqrt(Fy):
        Fs = 0.66*Fy
    elif hw <= 1435./sqrt(Fy):
        Fs = 670*sqrt(Fy)/hw
    else:
        Fs = 961200./(hw**2)
    Vr = phi*Aw*Fs * 1E-3
    if display:
        print('\nShear Strength:')
        show('h/w=hw,1014/sqrt(Fy),1435/sqrt(Fy),Fs,Aw,Vr',data=locals())
    
    
    return dict(Lu=sfround(Lu,3),L=L,Mr=sfround(Mr,3),Vr=sfround(Vr,3))

In [36]:
wl = SST.section_tables(['W','WWF'],'Mass')
s = SST.select(wl,check_Mr,maxn=10,Mf=Mf,L=5500.,omega2=1.75,Ireq=Ireq)
s[['Mass','Avl','Use','Vr','Ix','Lu','L','Mr']]

,Mass,Avl,Use,Vr,Ix,Lu,L,Mr
W610x113,113.0,*C,3.0,1400.0,8.750000e+08,3990.0,5500.0,906.0
W610x125,125.0,*C,3.0,1490.0,9.850000e+08,4100.0,5500.0,1030.0
W690x125,125.0,*,3.0,1610.0,1.190000e+09,4290.0,5500.0,1140.0
W760x134,134.0,*,3.0,1650.0,1.500000e+09,4340.0,5500.0,1320.0
W530x138,138.0,*C,3.0,1650.0,8.610000e+08,4080.0,5500.0,1020.0
W610x140,140.0,*C,3.0,1660.0,1.120000e+09,4210.0,5500.0,1180.0
W690x140,140.0,*,3.0,1740.0,1.360000e+09,4410.0,5500.0,1310.0
W760x147,147.0,*,3.0,2040.0,1.660000e+09,4380.0,5500.0,1460.0
W530x150,150.0,*,3.0,1410.0,1.010000e+09,6020.0,5500.0,1290.0
W690x152,152.0,*,3.0,1850.0,1.510000e+09,4500.0,5500.0,1450.0


In [37]:
s = SST.select(wl,check_Mr,maxn=10,Mf=Mf,L=5500.,omega2=1.75,Ireq=0)
s[['Mass','Avl','Use','Ix','Lu','L','Mr']]

,Mass,Avl,Use,Ix,Lu,L,Mr
W530x101,101.0,*C,3.0,6.170000e+08,3780.0,5500.0,706.0
W610x101,101.0,*C,3.0,7.640000e+08,3890.0,5500.0,784.0
W530x109,109.0,*C,3.0,6.670000e+08,3840.0,5500.0,772.0
W460x113,113.0,*,3.0,5.560000e+08,5430.0,5500.0,826.0
W610x113,113.0,*C,3.0,8.750000e+08,3990.0,5500.0,906.0
W410x114,114.0,*,3.0,4.680000e+08,5360.0,5500.0,768.0
W360x122,122.0,*,2.0,3.650000e+08,5900.0,5500.0,705.0
W530x123,123.0,*C,3.0,7.610000e+08,3950.0,5500.0,892.0
W610x125,125.0,*C,3.0,9.850000e+08,4100.0,5500.0,1030.0
W690x125,125.0,*,3.0,1.190000e+09,4290.0,5500.0,1140.0


In [38]:
s = SST.select(wl,check_Mr,maxn=10,Mf=Mf,L=4000.,omega2=1.,Ireq=Ireq) 
s[['Mass','Avl','Use','Ix','Lu','L','Mr']]

,Mass,Avl,Use,Ix,Lu,L,Mr
W610x113,113.0,*C,3.0,8.750000e+08,2950.0,4000.0,906.0
W610x125,125.0,*C,3.0,9.850000e+08,3020.0,4000.0,1020.0
W690x125,125.0,*,3.0,1.190000e+09,3190.0,4000.0,1140.0
W760x134,134.0,*,3.0,1.500000e+09,3230.0,4000.0,1330.0
W530x138,138.0,*C,3.0,8.610000e+08,2930.0,4000.0,1000.0
W610x140,140.0,*C,3.0,1.120000e+09,3070.0,4000.0,1170.0
W690x140,140.0,*,3.0,1.360000e+09,3270.0,4000.0,1320.0
W760x147,147.0,*,3.0,1.660000e+09,3260.0,4000.0,1470.0
W530x150,150.0,*,3.0,1.010000e+09,4380.0,4000.0,1290.0
W690x152,152.0,*,3.0,1.510000e+09,3320.0,4000.0,1460.0


In [39]:
SST.props.loc['Avl'].Desc

Availability, '*' indicates a section not produced in Canada. '+' indicates availability should be checked with supplier.

In [40]:
SST.get('W610x113').call(check_Mr,L=5500,omega2=1.75,display=True)

Section Properties and Dimensions:
Dsg = W610x113   
D   = 608        
B   = 228        
T   = 17.3       
W   = 11.2       
Ix  = 875 * 1E6  
Sx  = 2880 * 1E3 
Zx  = 3290 * 1E3 
Iy  = 34.3 * 1E6 
J   = 1120 * 1E3 
Cw  = 2990 * 1E9 
E   = 200000     
G   = 77000      
Fy  = 345        

Beam Data:
Ireq   = 0 * 1E6 
Mf     = 0       
L      = 5500    
omega2 = 1.75    

Section Class:
b/2t         = 6.59  
class 1      = 7.807 
class 2      = 9.152 
class 3      = 10.77 
flange class = 1     
h/w         = 51.2  
class 1     = 59.22 
class 2     = 91.52 
class 3     = 102.3 
web class   = 1     
sect. class = 1     

Moment Strength:
Mp      = 1135  
phi*Mp  = 1022  
L       = 5500  
Lu      = 3993  
Mu      = 1389  
0.67*Mp = 760.5 
Mr      = 905.9 

Shear Strength:
h/w           = 51.2  
1014/sqrt(Fy) = 54.59 
1435/sqrt(Fy) = 77.26 
Fs            = 227.7 
Aw            = 6810  
Vr            = 1395  


{Lu: 3990.0, L: 5500, Mr: 906.0, Vr: 1400.0}

In [41]:
SST.get('W610x113').call(check_Mr,L=3000,omega2=1,display=True)

Section Properties and Dimensions:
Dsg = W610x113   
D   = 608        
B   = 228        
T   = 17.3       
W   = 11.2       
Ix  = 875 * 1E6  
Sx  = 2880 * 1E3 
Zx  = 3290 * 1E3 
Iy  = 34.3 * 1E6 
J   = 1120 * 1E3 
Cw  = 2990 * 1E9 
E   = 200000     
G   = 77000      
Fy  = 345        

Beam Data:
Ireq   = 0 * 1E6 
Mf     = 0       
L      = 3000    
omega2 = 1       

Section Class:
b/2t         = 6.59  
class 1      = 7.807 
class 2      = 9.152 
class 3      = 10.77 
flange class = 1     
h/w         = 51.2  
class 1     = 59.22 
class 2     = 91.52 
class 3     = 102.3 
web class   = 1     
sect. class = 1     

Moment Strength:
Mp      = 1135  
phi*Mp  = 1022  
L       = 3000  
Lu      = 2952  
Mu      = 2363  
0.67*Mp = 760.5 
Mr      = 1017  

Shear Strength:
h/w           = 51.2  
1014/sqrt(Fy) = 54.59 
1435/sqrt(Fy) = 77.26 
Fs            = 227.7 
Aw            = 6810  
Vr            = 1395  


{Lu: 2950.0, L: 3000, Mr: 1020.0, Vr: 1400.0}